# Synthea Case Study
*Lab 26 April 2022*

* https://synthea.mitre.org/
* https://github.com/synthetichealth/synthea

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Patients;

CREATE TABLE Patients(
    id VARCHAR(36),
    birthdate DATE,
    deathdate DATE,
    ssn VARCHAR(11),
    drivers VARCHAR(9),
    passport VARCHAR(15),
    prefix VARCHAR(4),
    first VARCHAR(20),
    last VARCHAR(20),
    suffix VARCHAR(5),
    maiden VARCHAR(20),
    marital CHAR,
    race VARCHAR(10),
    ethnicity VARCHAR(15),
    gender CHAR,
    birthplace VARCHAR(100),
    address VARCHAR(50),
    city VARCHAR(30),
    state VARCHAR(30),
    county VARCHAR(50),
    zip VARCHAR(10),
    lat DECIMAL(18,15),
    lon DECIMAL(18,15),
    healthcare_expenses VARCHAR(20),
    healthcare_coverage VARCHAR(20),
    PRIMARY KEY(id)
) AS SELECT id,birthdate,deathdate,ssn,drivers,passport,prefix,first,last,suffix,maiden,marital,race,ethnicity,gender,
            birthplace,address,city,state,county,zip,lat,lon,healthcare_expenses,healthcare_coverage
FROM CSVREAD('../../data/synthea/scenario01/csv/patients.csv');

DROP TABLE IF EXISTS Encounters;

CREATE TABLE Encounters(
    id VARCHAR(36),
    start DATETIME,
    stop DATETIME,
    patient VARCHAR(36),
    organization VARCHAR(36),
    provider VARCHAR(36),
    payer  VARCHAR(36),
    encounterclass VARCHAR(20),
    code VARCHAR(15),
    description VARCHAR(100),
    base_encounter_cost DECIMAL(8,2),
    total_claim_cost DECIMAL(8,2),
    payer_coverage DECIMAL(8,2),
    reasoncode VARCHAR(15),
    reasondescription VARCHAR(100),
    PRIMARY KEY(id)
) AS SELECT id,start,stop,patient,organization,provider,payer,encounterclass,code,description,base_encounter_cost,
            total_claim_cost,payer_coverage,reasoncode,reasondescription
FROM CSVREAD('../../data/synthea/scenario01/csv/encounters.csv');

DROP TABLE IF EXISTS Conditions;

CREATE TABLE Conditions(
    start DATETIME,
    stop DATETIME,
    patient VARCHAR(36),
    encounter VARCHAR(36),
    code VARCHAR(20),
    description_condition VARCHAR(100),
) AS SELECT start,stop,patient,encounter,code,description
FROM CSVREAD('../../data/synthea/scenario01/csv/conditions.csv');

## Listing Patients

In [3]:
SELECT id, to_char(birthdate,'dd-mm-yyyy') birth, to_char(deathdate,'dd-mm-yyyy') death,
       deathdate-birthdate days_life
FROM Patients
LIMIT 5;

## Number of Dead Patients

In [4]:
SELECT COUNT(*) FROM Patients WHERE deathdate IS NOT NULL;

174

## Listing Encounters

In [5]:
SELECT id, to_char(start,'dd-mm-yyyy') start, encounterclass, description
FROM Encounters
LIMIT 5;

# Encounters in the Emergency

## Number of Patients with Cardiac Arrest

In [6]:
SELECT count(patient) FROM Encounters
       WHERE encounterclass = 'emergency' AND
       (description = 'Cardiac Arrest' OR description='Stroke' OR description='Myocardial Infarction');

119

## Number of Patients that Visited the Emergency

In [7]:
SELECT COUNT(DISTINCT patient) FROM Encounters
       WHERE encounterclass = 'emergency';

800

## Last Encounter in the Emergency

In [8]:
DROP VIEW IF EXISTS Emergency;

CREATE VIEW Emergency AS
SELECT patient, MAX(start) last_encounter
       FROM Encounters
       WHERE encounterclass = 'emergency'
       GROUP BY patient;

## Conditions in the Last Encounter

In [9]:
SELECT e.patient, c.description_condition
       FROM Emergency e, Conditions c
       WHERE e.patient = c.patient
       LIMIT 10;

## Patients that had a Stroke in the Last Encounter

In [10]:
SELECT COUNT(DISTINCT e.patient)
       FROM Emergency e, Conditions c
       WHERE e.patient = c.patient AND
            (c.description_condition='Cardiac Arrest' OR
             c.description_condition='Myocardial Infarction');

74

## Patients with a Stroke that Died 30 days after the Last Encounter

In [11]:
SELECT COUNT(*)
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 30 AND
             (c.description_condition='Cardiac Arrest' OR
              c.description_condition='Myocardial Infarction');

SELECT DISTINCT p.first, p.last,
                to_char(p.birthdate,'dd-mm-yyyy') birth, to_char(p.deathdate,'dd-mm-yyyy') death,
                (p.deathdate-p.birthdate)/365 age,
                to_char(s.last_encounter,'dd-mm-yyyy') last_encounter,
                p.deathdate-s.last_encounter days
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 30 AND
             (c.description_condition='Cardiac Arrest' OR
              c.description_condition='Myocardial Infarction');

## Conditions in the Last Encounter with Code

In [12]:
SELECT e.patient, c.code, c.description_condition
       FROM Emergency e, Conditions c
       WHERE e.patient = c.patient
       LIMIT 10;

## Patients that have Hypertension (code 59621000) in the Last Encounter

In [13]:
SELECT COUNT(DISTINCT e.patient)
       FROM Emergency e, Conditions c
       WHERE e.patient = c.patient AND c.code='59621000';

202

## Patients with Hypertension that died 30 days after the Last Encounter

In [14]:
SELECT COUNT(*)
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 30 AND
             c.code='59621000';

SELECT DISTINCT p.first, p.last,
                to_char(p.birthdate,'dd-mm-yyyy') birth, to_char(p.deathdate,'dd-mm-yyyy') death,
                (p.deathdate-p.birthdate)/365 age,
                to_char(s.last_encounter,'dd-mm-yyyy') last_encounter,
                p.deathdate-s.last_encounter days
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 30 AND
             c.code='59621000';

## Description of Conditions versus Encounters

In [15]:
SELECT DISTINCT c.description_condition, e.description
       FROM conditions c, encounters e
       WHERE c.encounter = e.id AND encounterclass = 'emergency';

## Description of Conditions

In [16]:
SELECT DISTINCT c.description_condition
       FROM conditions c, encounters e
       WHERE c.encounter = e.id AND encounterclass = 'emergency'
       ORDER BY c.description_condition;